# Análisis estático

# ARCHIVO 1 sample_vg655_25th.exe

### Realice un análisis de los ejecutables proporcionado. Responda las siguientes preguntas y muestre evidencia de los pasos:

### 1. ¿Se encuentran empaquetados los ejecutable? ¿Cómo lo puede determinar? (en caso positivo, desempaquetelos)

 argher    \Documents\uvg\Security_DS\act3\examples  ➜ (  main)   2ms   9:12 PM   
 ⚡arg ❯❯ upx -t sample_vg655_25th.exe 
                       Ultimate Packer for eXecutables
                          Copyright (C) 1996 - 2024
UPX 4.2.2       Markus Oberhumer, Laszlo Molnar & John Reiser    Jan 3rd 2024

upx: sample_vg655_25th.exe: NotPackedException: not packed by UPX

Tested 0 files.

- No esta empaquetado via UPX

In [1]:
import pefile
import capstone

In [2]:
pe = pefile.PE("./examples/sample_vg655_25th.exe")

# Verificar si hay UPX en las secciones
for section in pe.sections:
    print(f"Sección: {section.Name.decode().strip()}")
    print(f"  Tamaño en disco: {section.SizeOfRawData}")
    print(f"  Tamaño en memoria: {section.Misc_VirtualSize}")

    # Detectar UPX
    if b"UPX" in section.Name:
        print("Posible empaquetamiento con UPX detectado.")

# Verificar importaciones sospechosas
if not hasattr(pe, 'DIRECTORY_ENTRY_IMPORT') or len(pe.DIRECTORY_ENTRY_IMPORT) < 5:
    print("Pocas importaciones detectadas. Podría estar empaquetado.")


Sección: .text   
  Tamaño en disco: 28672
  Tamaño en memoria: 27056
Sección: .rdata  
  Tamaño en disco: 24576
  Tamaño en memoria: 24432
Sección: .data   
  Tamaño en disco: 8192
  Tamaño en memoria: 6488
Sección: .rsrc   
  Tamaño en disco: 3448832
  Tamaño en memoria: 3448736
Pocas importaciones detectadas. Podría estar empaquetado.


In [3]:
pe = pefile.PE("./examples/sample_vg655_25th.exe")

entry_point = pe.OPTIONAL_HEADER.AddressOfEntryPoint
image_base = pe.OPTIONAL_HEADER.ImageBase

code_section = None
for section in pe.sections:
    if entry_point >= section.VirtualAddress and entry_point < (section.VirtualAddress + section.Misc_VirtualSize):
        code_section = section
        break

if code_section:
    offset = entry_point - code_section.VirtualAddress
    raw_data = code_section.get_data()[offset:offset+32]  # Leer las primeras 32 bytes
    md = capstone.Cs(capstone.CS_ARCH_X86, capstone.CS_MODE_32)  # Para ejecutables de 32 bits

    print("Código ensamblador en el Entry Point:")
    for i in md.disasm(raw_data, image_base + entry_point):
        print(f"0x{i.address:x}: {i.mnemonic} {i.op_str}")


Código ensamblador en el Entry Point:
0x4077ba: push ebp
0x4077bb: mov ebp, esp
0x4077bd: push -1
0x4077bf: push 0x40d488
0x4077c4: push 0x4076f4
0x4077c9: mov eax, dword ptr fs:[0]
0x4077cf: push eax
0x4077d0: mov dword ptr fs:[0], esp
0x4077d7: sub esp, 0x68


### 2. Muestra una lista de las DLLs y funciones utilizadas. Lea el artículo Descargar artículo "Malware classification based on API calls and behaviour analysis." Compare la tabla 3 contra las funciones de los ejecutables. ¿Hay indicios de algún comportamiento malicioso en base a la comparación? Justifique su respuesta.

Se observaron estas actividades, en base a la tabla 3.

    Process Hollowing: CreateProcessA, GetModuleHandleA, GetProcAddress, VirtualAlloc son funciones clave para inyectar código en otro proceso.
    Drop de archivos desde la sección de recursos: FindResourceA, LoadResource, CreateFileA indican que el ejecutable podría estar extrayendo y escribiendo archivos desde su sección de recursos.
    IAT Hooking: GetModuleHandleA, strcmp, VirtualProtect podrían indicar modificación de la tabla de importaciones para ocultar funciones maliciosas.
    Autodestrucción: GetModuleFileNameA, ExitProcess podrían ser parte de un mecanismo de autodestrucción.

No hay evidencia directa de comunicación en red, pero esto no significa que el malware no tenga otras técnicas para evadir detección.


In [4]:
exe_path = "./examples/sample_vg655_25th.exe"  # Cambia por el nombre real del ejecutable

pe = pefile.PE(exe_path)

print("\nDLLs y funciones importadas:")
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(f"\n{entry.dll.decode()}")
    for imp in entry.imports:
        print(f"   └─ {imp.name.decode() if imp.name else 'ORDINAL ' + str(imp.ordinal)}")



DLLs y funciones importadas:

KERNEL32.dll
   └─ GetFileAttributesW
   └─ GetFileSizeEx
   └─ CreateFileA
   └─ InitializeCriticalSection
   └─ DeleteCriticalSection
   └─ ReadFile
   └─ GetFileSize
   └─ WriteFile
   └─ LeaveCriticalSection
   └─ EnterCriticalSection
   └─ SetFileAttributesW
   └─ SetCurrentDirectoryW
   └─ CreateDirectoryW
   └─ GetTempPathW
   └─ GetWindowsDirectoryW
   └─ GetFileAttributesA
   └─ SizeofResource
   └─ LockResource
   └─ LoadResource
   └─ MultiByteToWideChar
   └─ Sleep
   └─ OpenMutexA
   └─ GetFullPathNameA
   └─ CopyFileA
   └─ GetModuleFileNameA
   └─ VirtualAlloc
   └─ VirtualFree
   └─ FreeLibrary
   └─ HeapAlloc
   └─ GetProcessHeap
   └─ GetModuleHandleA
   └─ SetLastError
   └─ VirtualProtect
   └─ IsBadReadPtr
   └─ HeapFree
   └─ SystemTimeToFileTime
   └─ LocalFileTimeToFileTime
   └─ CreateDirectoryA
   └─ GetStartupInfoA
   └─ SetFilePointer
   └─ SetFileTime
   └─ GetComputerNameW
   └─ GetCurrentDirectoryA
   └─ SetCurrentDirectoryA

In [5]:
import datetime

# Extraer timestamp de compilación
timestamp = pe.FILE_HEADER.TimeDateStamp
compilation_date = datetime.datetime.utcfromtimestamp(timestamp)

# Mostrar resultado
print(f"📅 El ejecutable fue compilado el: {compilation_date}")


📅 El ejecutable fue compilado el: 2010-11-20 09:05:05


/tmp/ipykernel_13429/1544746521.py:5: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  compilation_date = datetime.datetime.utcfromtimestamp(timestamp)


### 3. ¿Cuándo fueron compilados los ejecutable?
En este caso, se observa que la fecha de compilacion es igual o mayor a 15 años.

### 4. Obtenga el código ensamblador aplicando ingeniería inversa de ambos ejecutables

In [6]:
entry_point = pe.OPTIONAL_HEADER.AddressOfEntryPoint
image_base = pe.OPTIONAL_HEADER.ImageBase

code_section = None
for section in pe.sections:
    if entry_point >= section.VirtualAddress and entry_point < (section.VirtualAddress + section.Misc_VirtualSize):
        code_section = section
        break

if code_section:
    offset = entry_point - code_section.VirtualAddress
    raw_data = code_section.get_data()[offset:offset+64]  # Leer primeras 64 bytes
    md = capstone.Cs(capstone.CS_ARCH_X86, capstone.CS_MODE_32)  # Cambia a CS_MODE_64 si es 64 bits

    print("\nCódigo ensamblador en el Entry Point:")
    for i in md.disasm(raw_data, image_base + entry_point):
        print(f"0x{i.address:x}: {i.mnemonic} {i.op_str}")



Código ensamblador en el Entry Point:
0x4077ba: push ebp
0x4077bb: mov ebp, esp
0x4077bd: push -1
0x4077bf: push 0x40d488
0x4077c4: push 0x4076f4
0x4077c9: mov eax, dword ptr fs:[0]
0x4077cf: push eax
0x4077d0: mov dword ptr fs:[0], esp
0x4077d7: sub esp, 0x68
0x4077da: push ebx
0x4077db: push esi
0x4077dc: push edi
0x4077dd: mov dword ptr [ebp - 0x18], esp
0x4077e0: xor ebx, ebx
0x4077e2: mov dword ptr [ebp - 4], ebx
0x4077e5: push 2
0x4077e7: call dword ptr [0x4081c4]
0x4077ed: pop ecx
0x4077ee: or dword ptr [0x40f94c], 0xffffffff


# ARCHIVO 2 sample_qwrty_dk2

In [7]:
pe = pefile.PE("./examples/sample_qwrty_dk2")


In [8]:
print('SECCIONES \t')
for section in pe.sections:
    print(section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize), section.SizeOfRawData)

SECCIONES 	
b'UPX0\x00\x00\x00\x00' 0x1000 0x5000 0
b'UPX1\x00\x00\x00\x00' 0x6000 0x1000 4096
b'.rsrc\x00\x00\x00' 0x7000 0x1000 512


In [9]:
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS A DLL \t')
    print (entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

LLAMADAS A DLL 	
b'KERNEL32.DLL'
LLAMADAS A FUNCIONES 	
	 b'LoadLibraryA'
	 b'ExitProcess'
	 b'GetProcAddress'
	 b'VirtualProtect'
LLAMADAS A DLL 	
b'MSVCRT.dll'
LLAMADAS A FUNCIONES 	
	 b'atol'
LLAMADAS A DLL 	
b'SHELL32.dll'
LLAMADAS A FUNCIONES 	
	 b'SHChangeNotify'
LLAMADAS A DLL 	
b'USER32.dll'
LLAMADAS A FUNCIONES 	
	 b'LoadStringA'
LLAMADAS A DLL 	
b'WS2_32.dll'
LLAMADAS A FUNCIONES 	
	 b'closesocket'


In [10]:
print("\nDLLs y funciones importadas:")
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(f"\n{entry.dll.decode()}")
    for imp in entry.imports:
        print(f"   └─ {imp.name.decode() if imp.name else 'ORDINAL ' + str(imp.ordinal)}")



DLLs y funciones importadas:

KERNEL32.DLL
   └─ LoadLibraryA
   └─ ExitProcess
   └─ GetProcAddress
   └─ VirtualProtect

MSVCRT.dll
   └─ atol

SHELL32.dll
   └─ SHChangeNotify

USER32.dll
   └─ LoadStringA

WS2_32.dll
   └─ closesocket


 argher    \Documents\uvg\Security_DS\act3\examples  ➜ (  main)   5ms   9:23 PM   
 ⚡arg ❯❯ upx -t sample_qwrty_dk2 
                       Ultimate Packer for eXecutables
                          Copyright (C) 1996 - 2024
UPX 4.2.2       Markus Oberhumer, Laszlo Molnar & John Reiser    Jan 3rd 2024

testing sample_qwrty_dk2 [OK]

Tested 1 file.

En este caso, si esta empaquetado

# Desempaquetado

In [13]:
import pefile

pe = pefile.PE("./examples/sample_qwrty_dk2_unpacked")

print("=== DLLs y Funciones Importadas ===")
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(f"[*] DLL: {entry.dll.decode()}")
    for imp in entry.imports:
        print(f"   - {imp.name.decode() if imp.name else 'Ordinal: ' + str(imp.ordinal)}")


=== DLLs y Funciones Importadas ===
[*] DLL: KERNEL32.DLL
   - CloseHandle
   - WaitForSingleObject
   - CreateEventA
   - ExitThread
   - Sleep
   - GetComputerNameA
   - CreatePipe
   - DisconnectNamedPipe
   - TerminateProcess
   - WaitForMultipleObjects
   - TerminateThread
   - CreateThread
   - CreateProcessA
   - DuplicateHandle
   - GetCurrentProcess
   - ReadFile
   - PeekNamedPipe
   - SetEvent
   - WriteFile
   - SetProcessPriorityBoost
   - SetThreadPriority
   - GetCurrentThread
   - SetPriorityClass
   - lstrcatA
   - lstrcpyA
   - GetEnvironmentVariableA
   - GetShortPathNameA
   - GetModuleFileNameA
   - GetStartupInfoA
   - GetModuleHandleA
[*] DLL: MSVCRT.dll
   - _controlfp
   - _beginthread
   - _strnicmp
   - sprintf
   - atol
   - strchr
   - free
   - malloc
   - _exit
   - _XcptFilter
   - exit
   - _acmdln
   - __getmainargs
   - _initterm
   - __setusermatherr
   - _adjust_fdiv
   - __p__commode
   - __p__fmode
   - __set_app_type
   - _except_handler3
   - _i

### 2. Muestra una lista de las DLLs y funciones utilizadas. Lea el artículo Descargar artículo "Malware classification based on API calls and behaviour analysis." Compare la tabla 3 contra las funciones de los ejecutables. ¿Hay indicios de algún comportamiento malicioso en base a la comparación? Justifique su respuesta.
Puntos en comun:

    - En este caso se presentaron CreateProcessA y CreateThread que pueden indicar intentos de inyección de procesos o ejecución de código malicioso.
    - ShellExecuteExA sugiere que el ejecutable podría descargar o ejecutar archivos.
    - Funciones de red (socket, WSAStartup, send, recv, connect) indican comunicación con servidores remotos, lo que puede estar relacionado con C2 (Command & Control) o exfiltración de datos.

In [15]:
from datetime import datetime

pe = pefile.PE("./examples/sample_qwrty_dk2_unpacked")
timestamp = pe.FILE_HEADER.TimeDateStamp
compilation_date = datetime.utcfromtimestamp(timestamp)

print(f"Fecha de compilación: {compilation_date}")


Fecha de compilación: 2009-05-14 17:12:40


/tmp/ipykernel_13429/2731632171.py:5: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  compilation_date = datetime.utcfromtimestamp(timestamp)


### 3. ¿Cuándo fueron compilados los ejecutable?

Este fue compilado el 2009, 1 año antes que el .exe que estaba en el .zip

### 4. Obtenga el código ensamblador aplicando ingeniería inversa de ambos ejecutables

In [16]:
import pefile
from capstone import *

entry_point = pe.OPTIONAL_HEADER.AddressOfEntryPoint
image_base = pe.OPTIONAL_HEADER.ImageBase
code_section = None

for section in pe.sections:
    if b".text" in section.Name:
        code_section = section
        break

if code_section:
    md = Cs(CS_ARCH_X86, CS_MODE_32)  # Cambia a CS_MODE_64 si es 64 bits
    code = code_section.get_data()
    offset = image_base + entry_point

    for i in md.disasm(code, offset):
        print(f"0x{i.address:x}:\t{i.mnemonic}\t{i.op_str}")


0xcc40ffee:	mov	eax, dword ptr [esp + 8]
0xcc40fff2:	mov	edx, dword ptr [esp + 4]
0xcc40fff6:	mov	ecx, eax
0xcc40fff8:	shr	ecx, 9
0xcc40fffb:	and	ecx, 1
0xcc40fffe:	xor	ecx, edx
0xcc410000:	je	0xcc410023
0xcc410002:	mov	ecx, eax
0xcc410004:	shr	ecx, 1
0xcc410006:	xor	ecx, eax
0xcc410008:	shr	ecx, 1
0xcc41000a:	xor	ecx, eax
0xcc41000c:	shr	ecx, 3
0xcc41000f:	xor	ecx, eax
0xcc410011:	and	eax, 0x3ffff
0xcc410016:	shr	ecx, 0xd
0xcc410019:	shl	eax, 1
0xcc41001b:	test	cl, 1
0xcc41001e:	je	0xcc410023
0xcc410020:	xor	eax, 1
0xcc410023:	ret	
0xcc410024:	nop	
0xcc410025:	nop	
0xcc410026:	nop	
0xcc410027:	nop	
0xcc410028:	nop	
0xcc410029:	nop	
0xcc41002a:	nop	
0xcc41002b:	nop	
0xcc41002c:	nop	
0xcc41002d:	nop	
0xcc41002e:	mov	eax, dword ptr [esp + 8]
0xcc410032:	mov	edx, dword ptr [esp + 4]
0xcc410036:	mov	ecx, eax
0xcc410038:	shr	ecx, 0xb
0xcc41003b:	and	ecx, 1
0xcc41003e:	xor	ecx, edx
0xcc410040:	je	0xcc410064
0xcc410042:	mov	ecx, eax
0xcc410044:	shr	ecx, 1
0xcc410046:	xor	ecx, eax
0xcc410048:	